In [14]:
import pandas as pd 
import numpy as np
import scipy as sp 
from sklearn.metrics.pairwise import cosine_similarity
import operator
import fastparquet as fp 
import pyarrow as pa 
import pyarrow.parquet as pq

In [27]:
df_item_rec = pd.read_parquet('../data/processed/df_item_rec.parquet')
df_item_rec.head()

,title,genres
id,,
10.0,Counter-Strike,Action
20.0,Team Fortress Classic,Action
30.0,Day of Defeat,Action
40.0,Deathmatch Classic,Action
50.0,Half-Life: Opposing Force,Action


In [28]:
#Se crean los dummies 
df_item_rec = pd.get_dummies(df_item_rec, columns=['genres'], prefix='')

df_item_rec = df_item_rec.groupby(['id', 'title']).sum().reset_index()

df_item_rec.head()

,id,title,_Accounting,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,_Design &amp; Illustration,_Early Access,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10.0,Counter-Strike,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20.0,Team Fortress Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30.0,Day of Defeat,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40.0,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50.0,Half-Life: Opposing Force,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
similitudes = cosine_similarity(df_item_rec.iloc[:,3:])
print(similitudes)

[[1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]]


In [31]:
similitudes.shape

(27422, 27422)

In [37]:
def recomendacion_juegos(id):
    
    id = int(id)
    # Filtrar el juego e igualarlo a  su ID
    juego_seleccionado = df_item_rec[df_item_rec['id'] == id]
    # devolver error en caso de vacio
    if juego_seleccionado.empty:
        return "El juego con el ID especificado no existe en la base de datos."
    
    # Calcula la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[df_item_rec[df_item_rec['id'] == id].index[0]]
    
    # Calcula los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]
    
    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = df_item_rec.iloc[indices_juegos_similares]['title']
    
    return juegos_recomendados

In [38]:
recomendacion_juegos(772540)

12658                Tempest: Pirate Action RPG
12701                  GGXrd System Voice - MAY
12700             GGXrd System Voice - KY KISKE
12699           GGXrd System Voice - SOL BADGUY
12698    GGXrd Character Unlock - LEO WHITEFANG
Name: title, dtype: object